In [2]:
import pandas as pd
import numpy as np
import re
import random

In [3]:
def ratelist(row_name, val_rmd):
    li = []
    
    if row_name == 'Q9_2A1':
        dic = {1.0:0.318198, 2.0:0.214775, 3.0:0.134685, 10.0:0.101622,
                4.0:0.092523, 6.0:0.064324, 5.0:0.034054,7.0:0.017477,
                8.0:0.013423, 9.0:0.008919}
    elif row_name == 'Q9_2A2':
        dic = {2.0:0.328732,1.0:0.192887,3.0:0.145144,4.0:0.110057,
                10.0:0.072189,5.0:0.066053,7.0:0.031636,6.0:0.030678,
                8.0:0.011888,9.0:0.010737}
    else:
        dic = {3.0:0.176084,2.0:0.157057,1.0:0.156711,4.0:0.129958,
                10.0:0.127076,5.0:0.111624,6.0:0.054659,7.0:0.047970,
                8.0:0.024331,9.0:0.014530}
    
    rate = list(dic.values())
    total = len(df18[df18[row_name]==21])
    count=0

    for i in range(1,11):
        num = int(round(dic[i],len(str(total)))*total)
        count+=num
        for j in range(0, num):
            li.append(i)
            
    if count != total:
        for k in range(0,total-count):
            li.append(int(list(dic.keys())[0]))
            
    random.shuffle(li)
    
    for l, m in zip(range(0,len(li)), df18[df18[row_name]==val_rmd].index):
        df18[row_name][m] = li[l]

In [4]:
df = pd.read_excel('데이터_변수정리_찐최종본.xlsx',
                    header =1,
                    usecols=[1,6,14],
                    sheet_name=0,
                    names = ['val1', 'val2', 'val3'])

val_lis = list(df['val3'].dropna(axis = 0))

val_lis2 = []
for val in val_lis:
    if '~' in val:
        text = re.compile('[0-9]~').search(val).start()
        i_start = re.compile('[a-zA-Z][0-9]{0,3}$').search(val).start()+1
        i_end = re.compile('[a-zA-Z][0-9]{0,3}$').search(val).end()
        ran = val[int(i_start):int(i_end)]
        for i in range(1,int(ran)+1):
            val_lis2.append('{}{}'.format(val[:text],i))
            
    else:
        val_lis2.append(val)

file_dir = '../01_raw데이터/'
df18 = pd.read_csv(file_dir+'2018년_원자료.csv')
df18 = df18.loc[:,val_lis2]
#df18

In [31]:
# 범주변경
val_lis=['Q2A1~Q2A5', 'Q5_2A1~Q5_2A3', 'Q9_2A1~Q9_2A3']
len_index = 0
df18['q2a1'] = np.nan
df18['q2a2'] = np.nan
df18['q2a3'] = np.nan
df18['q2a4'] = np.nan
df18['q2a5'] = np.nan
df18['Q2A1'] = df18['Q2A1'].fillna(1)

for val in val_lis:
    text = re.compile('[0-9]~').search(val).start()
    i_start = re.compile('[a-zA-Z][0-9]{0,3}$').search(val).start()+1
    i_end = re.compile('[a-zA-Z][0-9]{0,3}$').search(val).end()
    ran = val[int(i_start):int(i_end)]
    
    for i in range(1,int(ran)+1):
        name = '{}{}'.format(val[:text],i)
        
        if 'Q2A' in name:
            for j in range(0,len(df18)):
                if df18[name][j] == 1:
                    df18['q2a1'][j] = 1
                elif df18[name][j] == 2:
                    df18['q2a2'][j] = 2
                elif df18[name][j] == 3:
                    df18['q2a3'][j] = 3
                elif df18[name][j] == 4:
                    df18['q2a4'][j] = 4
                elif df18[name][j] == 5:
                    df18['q2a5'][j] = 5
        
        elif 'Q5_2A' in name:
            del_index = df18[df18[name]==4].index #del : 4:친구, 친지 방문
            len_index += len(del_index) #D_Q5_1 : 새 범주
            df18 = df18.drop(del_index)
            df18 = df18.reset_index(drop=True)
            df18[name] = df18[name].replace({5:4, 6:4, 8:5, 9:6, 10:8, 11:9,
                                             12:11, 13:12, 14:10, 15:3,
                                             16:13, 17:13})

        elif 'Q9_2A' in name:
            df18[name] = df18[name].replace({8:21}) #8임시값
            df18[name] = df18[name].replace({5: 8,7: 8,9: 5,10: 5,11: 7,12: 14,
                           13: 11,14: 12,15: 13,16: 9,17: 9,18: 10,
                           19: 10,20: 14})
            ratelist(name, 21) #함수 - 8 : 비율대로 분할

# D_Q5_1
df18['D_Q5_1'] = df18['D_Q5_1'].replace({2:3, 3:4, 4:5, 5:6})
i_dq5 = random.sample(list(df18[df18['D_Q5_1']==1].index),len_index)
df18.loc[i_dq5,'D_Q5_1'] = 2.0

# drop
df18 = df18.drop(['Q2A1','Q2A2','Q2A3','Q2A4','Q2A5'], axis=1)
df18.rename(columns={'q2a1':'Q2A1', 'q2a2':'Q2A2', 'q2a3':'Q2A3', 'q2a4':'Q2A4', 'q2a5':'Q2A5'},inplace=True)

In [33]:
df18_val = pd.read_excel('데이터_변수정리_찐최종본.xlsx',\
                           usecols = 'A : B', sheet_name = '18년여행지변수명')

for j in range(357) :
    df18 = df18.replace({'Q11A1' : df18_val.loc[j, '변수명']}, df18_val.loc[j, '변수내용'])
    df18 = df18.replace({'Q11A2' : df18_val.loc[j, '변수명']}, df18_val.loc[j, '변수내용'])
    df18 = df18.replace({'Q11A3' : df18_val.loc[j, '변수명']}, df18_val.loc[j, '변수내용'])

In [35]:
df18['Q14_2_89']=df18['Q14_2_8'] + df18['Q14_2_9']
df18['Q14_2_89_1'] = np.nan
i=0
for val in df18['Q14_2_89']:
    if val == 0:
        df18['Q14_2_89_1'][i] = 1
    elif val <=30:
        df18['Q14_2_89_1'][i] = 2
    elif val <=100:
        df18['Q14_2_89_1'][i] = 3
    elif val>10000:
        df18['Q14_2_89_1'][i] = 98
    else:
        df18['Q14_2_89_1'][i] = 4
    i+=1
df18 = df18.drop(['Q14_2_8','Q14_2_9','Q14_2_89'], axis=1)
df18.rename(columns={'Q14_2_89_1':'Q14_2_10'},inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 1
C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 2
C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 2
C:\Users\USER\AppData\Local\

C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 1
C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 2
C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 1
C:\Users\USER\AppData\Local\

C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 2
C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 2
C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 1
C:\Users\USER\AppData\Local\

C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 1
C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 1
C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 1
C:\Users\USER\AppData\Local\

C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 1
C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 1
C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 1
C:\Users\USER\AppData\Local\

C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 2
C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 1
C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 1
C:\Users\USER\AppData\Local\

C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 1
C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 2
C:\Users\USER\AppData\Local\Temp\ipykernel_12084\489162868.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df18['Q14_2_89_1'][i] = 3
C:\Users\USER\AppData\Local

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [37]:
df_col = pd.read_excel('데이터_변수정리_찐최종본.xlsx',
                    header =0,
                    usecols=[3,4],
                    sheet_name=4,
                    names = ['val18', 'val19'])
dic_col={}
for i in range(len(df_col)):
    dic_col.update({df_col['val18'][i]:df_col['val19'][i]})
df18.rename(columns=dic_col,inplace=True)

In [43]:
df18.to_excel('2018년_변수정리.xlsx', index=False)

In [48]:
pd.read_csv('2018년_변수정리.csv',encoding='euc-kr')

,Q1,Q5_2A1,Q5_2A2,Q5_2A3,Q9_2A1,Q9_2A2,Q9_2A3,Q10_1B1,Q10_1B2,Q10_1B3,...,Q15_2A8,Q15_2A10,Q15_2A12,Q16,Q2A1,Q2A2,Q2A3,Q2A4,Q2A5,Q14_2_10
0,1,1.0,2.0,4.0,5,1.0,4.0,1.0,NaN,NaN,...,5,2,3,2,1.0,NaN,NaN,NaN,NaN,1
1,1,3.0,NaN,NaN,1,2.0,NaN,1.0,NaN,NaN,...,5,5,5,4,NaN,2.0,NaN,NaN,NaN,2
2,6,5.0,1.0,2.0,1,2.0,NaN,1.0,NaN,NaN,...,4,2,4,4,1.0,NaN,NaN,NaN,NaN,2
3,3,1.0,2.0,5.0,1,2.0,9.0,1.0,NaN,NaN,...,5,5,5,5,NaN,NaN,3.0,NaN,NaN,2
4,1,6.0,NaN,NaN,5,4.0,1.0,1.0,NaN,NaN,...,5,4,5,4,1.0,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13776,1,1.0,3.0,12.0,3,2.0,4.0,NaN,2.0,NaN,...,5,4,4,4,NaN,NaN,3.0,NaN,NaN,2
13777,1,1.0,12.0,3.0,3,1.0,2.0,NaN,2.0,NaN,...,5,5,4,5,NaN,NaN,3.0,NaN,NaN,1
13778,1,4.0,12.0,3.0,3,1.0,4.0,NaN,2.0,NaN,...,5,5,0,5,NaN,NaN,3.0,NaN,NaN,1
13779,15,1.0,2.0,3.0,1,4.0,2.0,NaN,2.0,NaN,...,5,5,5,5,NaN,NaN,3.0,NaN,NaN,1
